In [ ]:
USE YourCompany_DW
GO

CREATE OR ALTER PROC BI.spETLFactAccountingVoucher2
(
	@PreYears INT
)
AS
BEGIN
	DECLARE @DateID INT

	SET @DateID = (SELECT
						(CalendarYear - @PreYears) * 10000 + 101
					FROM
						BI.DimDate
					WHERE
						GregorianDate = CONVERT(DATE, GETDATE())
				)


	/*
		این دستور معادل دستور بالا عمل می کند
		برای یادگیری هر دو را نوشتم
		SELECT
			@DateID = (CalendarYear - @PreYears) * 10000 + 101
		FROM
			BI.DimDate
		WHERE
			GregorianDate = CONVERT(DATE, GETDATE()
	*/
	
	--Delete Fact Table
	DELETE FROM BI.FactAccountingVoucher
	WHERE DateID >= @DateID


	--Merge Dimension Table
	MERGE INTO BI.DimAccount AS TARGET
	USING [YourCompany].BI.vwDimAccount AS SOURCE
	ON TARGET.AccountID = SOURCE.AccountID
	WHEN MATCHED AND (		
							TARGET.AccountCode <> SOURCE.AccountCode
						OR	TARGET.PrimaryCode <> SOURCE.PrimaryCode 
						OR	TARGET.GeneralCode <> SOURCE.GeneralCode 
						OR	TARGET.SubsidiaryCode <> SOURCE.SubsidiaryCode
						OR	TARGET.PrimaryTitle <> SOURCE.PrimaryTitle
						OR	TARGET.GeneralTitle <> SOURCE.GeneralTitle
						OR	TARGET.Title <> SOURCE.Title
						)
	THEN UPDATE
	SET 
		TARGET.AccountCode = SOURCE.AccountCode, 
		TARGET.PrimaryCode = SOURCE.PrimaryCode, 
		TARGET.GeneralCode = SOURCE.GeneralCode, 
		TARGET.SubsidiaryCode = SOURCE.SubsidiaryCode, 
		TARGET.PrimaryTitle = SOURCE.PrimaryTitle, 
		TARGET.GeneralTitle = SOURCE.GeneralTitle, 
		TARGET.Title = SOURCE.Title
	WHEN NOT MATCHED BY TARGET
	THEN INSERT 
	VALUES(
				SOURCE.AccountID, 
				SOURCE.AccountCode, 
				SOURCE.PrimaryCode, 
				SOURCE.GeneralCode, 
				SOURCE.SubsIDiaryCode, 
				SOURCE.PrimaryTitle, 
				SOURCE.GeneralTitle, 
				SOURCE.Title
			)
	WHEN NOT MATCHED BY SOURCE
	THEN DELETE;

	MERGE INTO BI.DimAccountingVoucherState AS TARGET
	USING [YourCompany].BI.vwDimAccountingVoucherState AS SOURCE
	ON 
		TARGET.AccountingVoucherStateID = SOURCE.AccountingVoucherStateID
	WHEN MATCHED
	THEN UPDATE
	SET 
		TARGET.Title = SOURCE.Title
	WHEN NOT MATCHED BY TARGET
	THEN INSERT
	VALUES
	(
		SOURCE.AccountingVoucherStateID, 
		SOURCE.Title
	)
	WHEN NOT MATCHED BY SOURCE
	THEN DELETE;


	--Insert Fact Table
	INSERT INTO BI.FactAccountingVoucher 
	SELECT 
		*
	FROM 
		YourCompany.BI.vwFactAccountingVoucher
	WHERE 
		DateID >= @DateID
END

In [ ]:
EXEC BI.spETLFactAccountingVoucher2 @PreYears = 1